In [1]:
#import relevant libraries

import pandas as pd
import json
from pandas.io.json import json_normalize

In [2]:
# Reading the json file as a dictionary

with open('./data/sample-ocds-award-data.json') as data:
    ocds_award = json.load(data)

In [3]:
# Printing out the keys of the sample ocds data

ocds_award.keys()

dict_keys(['uri', 'publishedDate', 'publisher', 'license', 'publicationPolicy', 'releases'])

In [4]:
# flattening out 'releases'

all_releases = json_normalize(ocds_award['releases'])
pd.DataFrame(all_releases).head()

,awards,buyer.address.countryName,buyer.address.locality,buyer.address.postalCode,buyer.address.region,buyer.address.streetAddress,buyer.contactPoint.email,buyer.contactPoint.faxNumber,buyer.contactPoint.name,buyer.contactPoint.telephone,...,buyer.identifier.legalName,buyer.identifier.scheme,buyer.identifier.uri,buyer.name,date,id,initiationType,language,ocid,tag
0,"[{'id': 'ocds-213czf-000-00001-award-01', 'tit...",United Kingdom,London,N11 1NP,London,"4, North London Business Park, Oakleigh Rd S",procurement-team@example.com,01234 345 345,Procurement Team,01234 345 346,...,London Borough of Barnet,GB-LAC,http://www.barnet.gov.uk/,London Borough of Barnet,2010-05-10T09:30:00Z,ocds-213czf-000-00001-04-award,tender,en,ocds-213czf-000-00001,[award]


In [5]:
# flattening out 'awards' to see all the details it contains

award_releases = json_normalize(ocds_award, 'releases', ['awards'], errors='ignore', record_prefix='awards/')
pd.DataFrame(award_releases).head()

,awards/awards,awards/buyer,awards/date,awards/id,awards/initiationType,awards/language,awards/ocid,awards/tag,awards
0,"[{'id': 'ocds-213czf-000-00001-award-01', 'tit...","{'identifier': {'scheme': 'GB-LAC', 'id': 'E09...",2010-05-10T09:30:00Z,ocds-213czf-000-00001-04-award,tender,en,ocds-213czf-000-00001,[award],NaN


In [6]:
#as we're still not getting supplier details let's further flatten 'awards/awards.'
#fist, let's read the column in a dataframe

award_details = pd.DataFrame(award_releases['awards/awards'])
award_details

,awards/awards
0,"[{'id': 'ocds-213czf-000-00001-award-01', 'tit..."


In [7]:
#unpacking all details in the new dataframe 'award_details'

def unpack(award_details):
    award_details_unpacked = []
    for i in award_details:
        if type(i) != list:
            award_details_unpacked.append(i)
        else:
            award_details_unpacked = award_details_unpacked + unpack(i)
    return award_details_unpacked

award_details_unpacked = {}
for k, v in award_details.items():
    award_details_unpacked[k] = unpack(v)

In [8]:
#printing out unpacked dataframe

award_details_unpacked

{'awards/awards': [{'id': 'ocds-213czf-000-00001-award-01',
   'title': 'Award of contract to build new cycle lanes in the centre of town.',
   'description': 'AnyCorp Ltd has been awarded the contract to build new cycle lanes in the centre of town.',
   'status': 'pending',
   'date': '2010-05-10T09:30:00Z',
   'value': {'amount': 11000000, 'currency': 'GBP'},
   'suppliers': [{'identifier': {'scheme': 'GB-COH',
      'id': '1234567844',
      'legalName': 'AnyCorp Ltd',
      'uri': 'http://www.anycorp.example'},
     'additionalIdentifiers': [{'scheme': 'GB-VAT', 'id': '987654321'}],
     'name': 'AnyCorp Cycle Provision',
     'address': {'streetAddress': '100 Anytown Lane',
      'locality': 'Anytown',
      'region': 'AnyCounty',
      'postalCode': 'AN1 1TN',
      'countryName': 'United Kingdom'},
     'contactPoint': {'name': 'Contracts Team',
      'email': 'contracts@anycorp.example',
      'telephone': '12345 456 343',
      'faxNumber': '12345 456 343'}}],
   'items': [{'i